In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark =SparkSession.builder.appName('3thanwy').getOrCreate()

In [ ]:
import pandas as pd

In [ ]:
dff=pd.read_excel("/نتيجة الثانوية العامة 2025.xlsx")

In [ ]:
dff.head()

,seating_no,arabic_name,total_degree
0,1001660,محمد ابو الحسن حسن مصطفى,163.5
1,1001661,محمد احمد محمد ابو زيد,187.5
2,1001662,محمد على محمود عبدالعزيز,168.0
3,1001663,محمود سيد انور محمد حامد,212.0
4,1001664,محمود عطيه محمود جابر حجاج,154.0


In [ ]:
df = spark.createDataFrame(dff)

In [ ]:
df1=df.na.drop(subset=["total_degree"])

In [ ]:
df1.filter(col("total_degree").isNull()).show()

+----------+-----------+------------+
|seating_no|arabic_name|total_degree|
+----------+-----------+------------+
+----------+-----------+------------+



In [ ]:
df1.printSchema()

root
 |-- seating_no: long (nullable = true)
 |-- arabic_name: string (nullable = true)
 |-- total_degree: double (nullable = true)



In [ ]:
df_mohafza= df1.withColumn("mohafza_code",substring(col("seating_no"),0,2))

In [ ]:
df_mohafza.groupBy("mohafza_code").agg(count("*").alias("students")).show()

+------------+--------+
|mohafza_code|students|
+------------+--------+
|          11|   79702|
|          10|   92629|
|          12|   88448|
|          13|   89465|
|          14|   91900|
|          15|   86614|
|          16|   90694|
|          18|   89806|
|          17|   85690|
|          19|   16032|
+------------+--------+



In [ ]:
df_total_mya=df_mohafza.withColumn("total_degree_100",col("total_degree")*100/320)

In [ ]:
df_total_mya.show(3)

+----------+--------------------+------------+------------+----------------+
|seating_no|         arabic_name|total_degree|mohafza_code|total_degree_100|
+----------+--------------------+------------+------------+----------------+
|   1001660|محمد ابو الحسن حس...|       163.5|          10|        51.09375|
|   1001661|محمد احمد محمد اب...|       187.5|          10|        58.59375|
|   1001662|محمد على محمود عب...|       168.0|          10|            52.5|
+----------+--------------------+------------+------------+----------------+
only showing top 3 rows



In [ ]:
df_sucess=df_total_mya.filter(col("total_degree_100")>=50)
df_fail=df_total_mya.filter(col("total_degree_100")<50)


In [ ]:
df.agg(count("*")).show()
df_sucess.agg(count("*")).show()
df_fail.agg(count("*")).show()

+--------+
|count(1)|
+--------+
|  810980|
+--------+

+--------+
|count(1)|
+--------+
|  689444|
+--------+

+--------+
|count(1)|
+--------+
|  121536|
+--------+



In [ ]:
df_sucess.groupBy("mohafza_code").agg(count("*").alias("sucess_students")).orderBy("sucess_students",ascending=False).show()

+------------+---------------+
|mohafza_code|sucess_students|
+------------+---------------+
|          16|          87112|
|          14|          83117|
|          18|          77911|
|          17|          75505|
|          15|          74536|
|          13|          74141|
|          12|          72116|
|          10|          68811|
|          11|          61806|
|          19|          14389|
+------------+---------------+



In [ ]:
df_sucess.filter(col("total_degree_100")>=90).groupBy("mohafza_code").agg(count("*").alias("sucess_students")).orderBy("sucess_students",ascending=False).show()

+------------+---------------+
|mohafza_code|sucess_students|
+------------+---------------+
|          18|          11724|
|          16|           9573|
|          17|           6480|
|          14|           6273|
|          15|           4752|
|          12|           4446|
|          11|           4287|
|          19|           4012|
|          13|           3844|
|          10|           3626|
+------------+---------------+



In [ ]:
df_sucess_cat=df_sucess.withColumn("category", when((col("total_degree_100") >= 50) & (col("total_degree_100") < 60), "50-60")
    .when((col("total_degree_100") >= 60) & (col("total_degree_100") < 70), "60-70")
    .when((col("total_degree_100") >= 70) & (col("total_degree_100") < 80), "70-80")
    .when((col("total_degree_100") >= 80) & (col("total_degree_100") < 90), "80-90")
    .when((col("total_degree_100") >= 90) & (col("total_degree_100") <= 100), "90-100")
    )

In [ ]:
df_sucess_cat.groupBy("category").agg(count("*").alias("sucess_students")).orderBy("sucess_students",ascending=False).show()

+--------+---------------+
|category|sucess_students|
+--------+---------------+
|   60-70|         188224|
|   70-80|         166925|
|   80-90|         140065|
|   50-60|         135213|
|  90-100|          55422|
|    NULL|           3595|
+--------+---------------+



In [ ]:
#partining
df_partitioned = df_mohafza.repartition("mohafza_code")


In [ ]:
df_mohafza.groupBy("mohafza_code").agg(count("*").alias("students")).show(2)


+------------+--------+
|mohafza_code|students|
+------------+--------+
|          11|   79702|
|          10|   92629|
+------------+--------+
only showing top 2 rows



In [ ]:
df_partitioned.groupBy("mohafza_code").agg(count("*").alias("students")).show(2)

+------------+--------+
|mohafza_code|students|
+------------+--------+
|          11|   79702|
|          10|   92629|
+------------+--------+
only showing top 2 rows

